In [ ]:
# develop-activating-states-function.ipynb
#
# Bryan Daniels
# 2024/1/8
#
# As part of an effort to find all basins of size 1, we need a function that takes a 
# Neet network and a node index and produces the states that activate or deactivate 
# that node.
#

For `LogicNetwork`s, this is easy, because we already store the network's logic table in a similar format.

For `WTNetwork`s, we will just want to analyze all possible states of each node's inputs.

In [1]:
from neet import UniformSpace

def activating_states(net,node_index,activate=True):
    """
    Returns the states that activate or deactivate a given node in a Neet network.
    
    Returns: (neighbors_tuple,conditions),
    where node_tuple is a sorted tuple of node indices on which the given node depends, 
    and conditions is a tuple of binary states that activate the given node
    (or deactivate the given node if activate=False)
    """
    neighbors_list = tuple(sorted(net.neighbors_in(node_index)))
    conditions = []
    default_state = [ 0 for i in range(net.size) ] # states to which other non-input nodes are set
    input_space = UniformSpace(len(neighbors_list),2) # create binary input space
    for reduced_state in input_space:
        
        # set starting state to have given values for input nodes
        full_state = default_state.copy()
        for i,neighbor in enumerate(neighbors_list):
            full_state[neighbor] = reduced_state[i]
        
        # check whether this state is activating for node_index
        if net.update(full_state)[node_index] == activate:
            conditions.append(tuple(reduced_state))
            
    return neighbors_list,tuple(conditions)

### checks

In [2]:
# load/create some example networks
from neet.boolean.examples import mouse_cortical_7B # a logic network
from neet.boolean.examples import c_elegans # a weight-threshold network
from neet.boolean.examples import hgf_signaling_in_keratinocytes # a larger logic network
from neet.boolean import LogicNetwork
test_table = [((1,0),{'10','11'}),
              ((1,),{'1'})]
test_net = LogicNetwork(test_table)

In [3]:
import numpy as np

# check that the sum of activating and deactivating states is 2^(number of inputs)
def check_sum(net):
    for i in range(net.size):
        neighbors,act_conditions = activating_states(net,i,True)
        neighbors,deact_conditions = activating_states(net,i,False)
        assert(len(act_conditions) + len(deact_conditions) == 2**(len(neighbors)))
        
# check that a logic table network has the same inputs and activating conditions
def check_logic(net):
    for i in range(net.size):
        neighbors,act_conditions = activating_states(net,i)
        
        # check that neighbors are the same
        assert(tuple(sorted(net.table[i][0])) == neighbors)
        
        # check that length of activating conditions is the same
        assert(len(net.table[i][1]) == len(act_conditions))
        
        # check that activating conditions are the same
        if len(neighbors) > 0:
            table_conditions = [ np.array([int(val) for val in list(condition)]) for condition in net.table[i][1] ]
            sorted_table_conditions = [ tuple(condition[np.argsort(net.table[i][0])]) \
                                             for condition in table_conditions ]
            assert(set(sorted_table_conditions) == set(act_conditions))

In [4]:
check_sum(c_elegans)
check_sum(mouse_cortical_7B)
check_sum(hgf_signaling_in_keratinocytes)
check_sum(test_net)

In [5]:
check_logic(mouse_cortical_7B)
check_logic(hgf_signaling_in_keratinocytes)
check_logic(test_net)

In [6]:
mouse_cortical_7B.table

[((9,), {'1'}),
 ((5, 7), {'00'}),
 ((5, 8), {'10'}),
 ((5, 7, 9), {'000'}),
 ((8, 6), {'00'}),
 ((0,), {'1'}),
 ((1,), {'1'}),
 ((2,), {'1'}),
 ((3,), {'1'}),
 ((4,), {'1'})]

In [7]:
activating_states(mouse_cortical_7B,1,True)

((5, 7), ((0, 0),))

In [8]:
activating_states(hgf_signaling_in_keratinocytes,0)

((33, 40, 43),
 ((1, 0, 0), (0, 1, 0), (1, 1, 0), (0, 0, 1), (1, 0, 1), (0, 1, 1), (1, 1, 1)))

In [9]:
activating_states(test_net,0)

((0, 1), ((0, 1), (1, 1)))